In [ ]:
!pip install openai langchain llama-index google-search-results faiss-cpu

In [ ]:
import os
# 環境変数
os.environ["OPENAI_API_KEY"] = "Your Open API KEY"

In [ ]:
SERP_API_KEY = "Your Serp API KEY"

In [ ]:
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

# 検索するAPIのラッパーを用意する
search = SerpAPIWrapper(
    serpapi_api_key=SERP_API_KEY,
    params = {"engine": "bing","gl": "jp","hl": "ja"}
)

# ツールにエージェントが利用する機能を格納する
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Serch_trash_day",
        func=day_qa_chain.run,
        description="useful if you want to find out what day to dump that \
        type of trash in Bukyo-ku."
    )
]

In [ ]:
# エージェントが利用するプロンプトを作成する
template = """Answer the following questions.You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer (in Japanese): the final answer to the original input question in Japanese
Question: {input}
{agent_scratchpad}"""

In [ ]:
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import HumanMessage
from typing import List, Union

# エージェントのプロンプトのテンプレートを定義する
class CustomPromptTemplate(BaseChatPromptTemplate):
    template: str
    tools: List[Tool]
        
    def format_messages(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, HumanMessage

# エージェントの出力のパーサーを作成する
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer (in Japanese):" in llm_output:
            return AgentFinish(
                return_values={
                    "output": llm_output.split("Final Answer (in Japanese):")[-1].strip()
                },
                log=llm_output,
            )
        # 行動と行動のための入力をパースする
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # 行動と行動のための入力を返す
        return AgentAction(tool=action,
                           tool_input=action_input.strip(" ").strip('"'),
                           log=llm_output)

output_parser = CustomOutputParser()

In [ ]:
from langchain.agents import LLMSingleActionAgent
from langchain.agents import AgentExecutor

# エージェントを設定する
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                tools=tools,
                verbose=True)

In [ ]:
agent_executor.run("ここに質問を記入")